In [1]:
import pandas as pd
import ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots


class Torgash:
    def __init__(self):
        self.current_base_balance = 90000000
        self.base_symbol = 'usdt'
        self.current_trading_balance = 0
        self.trading_symbol = 'eth'
        self.trading_fee_multiplier = 1 - 0.0002
        self.min_bid_threshold = 0
        self.min_ask_threshold = 0
        ###################################
        self.transaction_type_hist = []
        self.transaction_amount_hist = []
        self.transaction_balance_hist = []
        self.transaction_datetime = []
        ###################################
        self.base_balance_hist = []
        self.trading_balance_hist = []
        ###################################
        self.data = None
        self.ind = None
        self.current_datetime = None
        self.current_price = 0
        ###################################
        order_list = []

    def set_data(self, ohlcv_df: pd.DataFrame, date_time=0, open=1, high=2, low=3, close=4, volume=5):
        self.data = pd.DataFrame([
            ohlcv_df[ohlcv_df.columns[date_time]],
            ohlcv_df[ohlcv_df.columns[open]],
            ohlcv_df[ohlcv_df.columns[high]],
            ohlcv_df[ohlcv_df.columns[low]],
            ohlcv_df[ohlcv_df.columns[close]],
            ohlcv_df[ohlcv_df.columns[volume]]]).T
        self.data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
        try:
            self.data['Datetime'] = pd.to_datetime(self.data['Datetime'], unit='ms')
        except:
            raise ValueError("Datetime column format must be unix or string format.")
        ##################################################### calculate indicators to dataframe
        self.data = self.data.join(ta.trend.sma_indicator(self.data.Close, window=50))
        self.data = self.data.join(ta.trend.sma_indicator(self.data.Close, window=100))
        self.data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'sma_fast',
                             'sma_slow']  # name the columns the way you want
        #####################################################
        self.data.set_index('Datetime', inplace=True)

    def set_data_from_csv(self, filename: str, date_time=0, open=1, high=2, low=3, close=4, volume=5):
        ohlcv_df = pd.read_csv(filename)
        self.set_data(ohlcv_df, date_time=date_time, open=open, high=high, low=low, close=close, volume=volume)

    def buy(self, amount=-1., ):
        if amount < 0:
            amount = self.current_base_balance
        if self.current_base_balance >= amount >= self.min_bid_threshold:
            self.current_base_balance -= amount
            self.current_trading_balance += (amount / self.current_price) * self.trading_fee_multiplier
            ###
            self.transaction_datetime.append(self.current_datetime)
            self.transaction_type_hist.append('buy')
            self.transaction_balance_hist.append(
                self.current_base_balance + (self.current_trading_balance * self.current_price))
            self.transaction_amount_hist.append(amount)
            ###
            print(f"|| {self.current_datetime} | Buy | {amount} {self.base_symbol} --> "
                  f"{(amount / self.current_price) * self.trading_fee_multiplier} {self.trading_symbol} || "
                  f"Balance: {self.current_base_balance} {self.base_symbol} | "
                  f"{self.current_trading_balance} {self.trading_symbol} ||")
            self.print_ind()  # xtra output for necessary indicators
        else:
            print("Buy attempt, but not enough money.")

    def sell(self, amount=-1., ):
        if amount < 0:
            amount = self.current_trading_balance
        if self.current_trading_balance >= amount >= self.min_ask_threshold:
            self.current_trading_balance -= amount
            self.current_base_balance += (amount * self.current_price) * self.trading_fee_multiplier
            ###
            self.transaction_datetime.append(self.current_datetime)
            self.transaction_type_hist.append('sell')
            self.transaction_balance_hist.append(
                self.current_base_balance + (self.current_trading_balance * self.current_price))
            self.transaction_amount_hist.append(amount)
            ###
            print(f"|| {self.current_datetime} | Sell | {amount} {self.trading_symbol} --> "
                  f"{(amount * self.current_price) * self.trading_fee_multiplier} {self.base_symbol} || "
                  f"Balance: {self.current_base_balance} {self.base_symbol} | "
                  f"{self.current_trading_balance} {self.trading_symbol} ||")
            self.print_ind()  # xtra output for necessary indicators
        else:
            print("Sell attempt, but not enough money.")

    def print_ind(self):  # put indicators u need (ex: )
        print("")

    def run_strategy(self):
        self.data = self.data.iloc[
                    99:]  # We cut the top of the data, because first 100 sma values are not calculated there.
        ##################################################### xtra variables
        fast_above_slow = self.data.sma_fast[self.data.head(1).index[0]] > self.data.sma_slow[
            self.data.head(1).index[0]]
        #####################################################
        self.base_balance_hist.append(self.current_base_balance)
        self.trading_balance_hist.append(self.current_trading_balance)
        for datetime, value in self.data.iterrows():
            self.current_datetime = datetime
            self.current_price = value.Close
            ##################################################### There you should implement your strategy
            if (fast_above_slow == True) and (self.data.sma_fast[datetime] < self.data.sma_slow[datetime]):
                self.sell()
            elif (fast_above_slow == False) and (self.data.sma_fast[datetime] > self.data.sma_slow[datetime]):
                self.buy()
            fast_above_slow = self.data.sma_fast[datetime] > self.data.sma_slow[datetime]
            #####################################################
            self.base_balance_hist.append(self.current_base_balance)
            self.trading_balance_hist.append(self.current_trading_balance)

    def plot(self):
        i = 0
        count_candlestick_per_plot = 10000

        transaction_balance_df = pd.DataFrame(
            [vasya.transaction_datetime, vasya.transaction_balance_hist, vasya.transaction_type_hist]).T
        data_interval = self.data[i:i + count_candlestick_per_plot]
        transaction_balance_df_interval = transaction_balance_df[transaction_balance_df[0].between(data_interval.index.values[0], data_interval.index.values[-1])]
        main = make_subplots(rows=2, cols=1, row_width=[0.3, 0.7])

        main.add_trace(go.Candlestick(x=data_interval.index.values,
                                      open=data_interval['Open'], high=data_interval['High'],
                                      low=data_interval['Low'], close=data_interval['Close'],
                                      name=f'{self.base_symbol}/{self.trading_symbol}'), row=1, col=1)

        main.add_trace(
            go.Scatter(x=transaction_balance_df_interval[0], y=transaction_balance_df_interval[1], name='balance'),
            row=2, col=1)
        main.add_trace(go.Scatter(mode='markers',
                                  x=transaction_balance_df_interval[transaction_balance_df_interval[2] == 'buy'][0],
                                  y=transaction_balance_df_interval[transaction_balance_df_interval[2] == 'buy'][1],
                                  marker_symbol=45,
                                  marker_color='green', name='buy'), row=2, col=1)
        main.add_trace(go.Scatter(mode='markers',
                                  x=transaction_balance_df_interval[transaction_balance_df_interval[2] == 'sell'][0],
                                  y=transaction_balance_df_interval[transaction_balance_df_interval[2] == 'sell'][1],
                                  marker_symbol=46,
                                  marker_color='red', name='sell'), row=2, col=1)

        main.update_layout(xaxis_rangeslider_visible=False, height=800)
        main.show()

        #coming soon...


In [2]:

data = pd.read_csv("eth_usdt_1m_2022_01_01.csv")
data = data.iloc[:10000]

FileNotFoundError: [Errno 2] No such file or directory: 'eth_usdt_1m_2022_01_01.csv'

In [ ]:
vasya = Torgash()

vasya.set_data(data)

In [ ]:
vasya.run_strategy()

In [ ]:
vasya.data.head(1000)

In [ ]:
vasya.data.sma_fast[vasya.data.head(1).index[0]]

In [ ]:
vasya.plot()

In [ ]:

time_balance_df = pd.DataFrame([vasya.data.index, vasya.base_balance_hist, vasya.trading_balance_hist]).T

time_balance_df.set_index(0, inplace=True)
time_balance_df[2] = time_balance_df[2] * vasya.data['Close']
time_balance_df[1] = time_balance_df[1].apply(lambda x: None if x == 0.0 else x)
time_balance_df[2] = time_balance_df[2].apply(lambda x: None if x == 0.0 else x)
time_balance_df = time_balance_df.reset_index(level=0)
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_balance_df[0], y=time_balance_df[1]))
fig.add_trace(go.Scatter(x=time_balance_df[0], y=time_balance_df[2]))
fig.show()

In [ ]:
#transaction_balance_df = pd.DataFrame(
#    [vasya.transaction_datetime, vasya.transaction_balance_hist, vasya.transaction_type_hist]).T
#
#transaction_balance_fig = go.Figure()
#transaction_balance_fig.add_trace(go.Scatter(x=transaction_balance_df[0], y=transaction_balance_df[1], name='balance'))
#transaction_balance_fig.add_trace(go.Scatter(mode='markers', x=transaction_balance_df[transaction_balance_df[2] == 'buy'][0],
#                         y=transaction_balance_df[transaction_balance_df[2] == 'buy'][1], marker_symbol=45,
#                         marker_color='green', name='buy'))
#transaction_balance_fig.add_trace(go.Scatter(mode='markers', x=transaction_balance_df[transaction_balance_df[2] == 'sell'][0],
#                         y=transaction_balance_df[transaction_balance_df[2] == 'sell'][1], marker_symbol=46,
#                         marker_color='red', name='sell'))
#transaction_balance_fig.show()